# G2Config Reference

More information:

1. [GitHub repository](https://github.com/Senzing/docker-jupyter)
1. [Senzing documentation](http://docs.senzing.com/?python#g2config)

## Table of contents

1. [Prepare environment](#Prepare-environment)
    1. [Helper class for JSON rendering](#Helper-class-for-JSON-rendering)
    1. [System path](#System-path)
    1. [Initialize variables](#Initialize-variables)
1. [G2Config](#G2Config)
    1. [G2Config initialization](#G2Config-initialization)
    1. [listDataSources](#listDataSources)
    1. [save](#save)
    1. [create](#create)

## Prepare environment

In [ ]:
import os
import sys
import json

# For RenderJSON

import uuid
from IPython.display import display_javascript, display_html, display

### Helper class for JSON rendering

A class for pretty-printing JSON.
Not required by Senzing, 
but helps visualize JSON.

In [ ]:
class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        elif isinstance(json_data, bytearray):
            self.json_str = json_data.decode()
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height:100%; width:100%; background-color: LightCyan"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

### System path

Update system path.

In [ ]:
python_path = "{0}/python".format(
    os.environ.get("SENZING_G2_DIR", "/opt/senzing/g2"))
sys.path.append(python_path)

### Initialize variables

Create variables used for G2Engine.

In [ ]:
module_name = 'pyG2Config'

config_path = os.environ.get("SENZING_ETC_DIR", "/etc/opt/senzing")
support_path = os.environ.get("SENZING_DATA_VERSION_DIR", "/opt/senzing/data")

resource_path = "{0}/resources".format(
    os.environ.get("SENZING_G2_DIR", "/opt/senzing/g2"))

sql_connection = os.environ.get(
    "SENZING_SQL_CONNECTION", "sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db")

verbose_logging = False

senzing_config_dictionary = {
    "PIPELINE": {
        "CONFIGPATH": config_path,        
        "SUPPORTPATH": support_path,
        "RESOURCEPATH": resource_path
    },
    "SQL": {
        "CONNECTION": sql_connection,
    }
}

senzing_config_json = json.dumps(senzing_config_dictionary)

## G2Config

The G2Config API is used to create a new JSON configuration from the default template
or manipulate a JSON configuration that it is provided.
The API read/writes a JSON string, but that string would typically be
read/written from/to a g2config.json file that the G2 engine reads during the initV2(...) call.

In [ ]:
from G2Config import G2Config

### G2Config Initialization

To start using G2Config, you must first create and initialize the G2Config object.
This should be done once per process.

Create a new instance of the Senzing G2Config object and assign it to a variable.
Then, call the appropriate initialization method (such as initV2) to initialize the G2Config object.

During the call, the initialization methods accept the following parameters:

- **module_name:** A short name given to this instance of the G2 engine (i.e. your G2Module object)
- **senzing_config_json:** A JSON document containing system parameters (see the section called "Initialization Parameters")
- **verbose_logging:** A boolean which enables diagnostic logging - this will print a massive amount of information to stdout (default = False)

Calling these functions will return "0" upon success - useful for error handling.

In [ ]:
g2_config = G2Config()
return_code = g2_config.initV2(module_name, senzing_config_json, verbose_logging)

print("Return Code: {0}".format(return_code))

### create

The `create()` method creates a stock Senzing JSON config from the template in `${SUPPORTPATH}/g2config.json`.
Importantly, the `create()` method returns a long integer referred to here as a `config_handle`.
This `config_handle` is a handle referring to the created Senzing configuration,
and can be used to represent said Senzing configuration in later function calls.

In [ ]:
config_handle = g2_config.create()

### listDataSources

The `listDataSources()` method returns a list of the G2Config object's current data sources
as a JSON document to a user-designated string.
The `listDataSources()` method returns a 0 upon success, and fails otherwise.
The `listDataSources()` method accepts the following parameters:

- **config_handle:** A long integer representing a previously created G2Config object
- **response_bytearray:** A memory buffer for returning the response document; if an error occurred, an error response is stored here

In [ ]:
response_bytearray = bytearray()

return_code = g2_config.listDataSources(config_handle, response_bytearray)

print("Return Code: {0}".format(return_code))
RenderJSON(response_bytearray)

### save

The `save()` method saves the G2Config object into a JSON string.
The `save()` method returns 0 upon success,
and it accepts the following parameters:

- **config_handle:** A long integer representing a previously created G2Config object
- **response_bytearray:** A memory buffer for returning the response document; if an error occurred, an error response is stored here


In [ ]:
response_bytearray = bytearray()

return_code = g2_config.save(config_handle, response_bytearray)

print("Return Code: {0}".format(return_code))
RenderJSON(response_bytearray)